In [ ]:
# Imports
from PIL import Image, ImageFilter
import numpy as np
from transformers import pipeline
import gradio as gr

model = pipeline("image-segmentation")

def image_objects(image):
    pred = model(image)
    pred_object_list = [x['label'] for x in pred]
    return gr.Dropdown.update(choices = pred_object_list, interactive = True)


def Recommend_new(x):
    return x + ": Highest Cosine Similarity"

app = gr.Blocks()

options = ['Placeholder A', 'Placeholder B', 'Placeholder C']
with app:
    image_input = gr.Image(type="pil")

    text_options = gr.Dropdown()
    b2 = gr.Button("Provide Additional options")
    
    new_title = gr.Textbox(label="Here you go!")
    b3 = gr.Button("Recommend a new title")

    b2.click(image_objects, inputs=image_input, outputs=text_options)
    # b3.click(Recommend_new, inputs=text_options, outputs=new_title)


app.launch(debug=True)

In [ ]:
# Imports
from PIL import Image, ImageFilter
import numpy as np
from transformers import pipeline
import gradio as gr

model = pipeline("image-segmentation")

def image_objects(image):
    pred = model(image)
    pred_object_list = [x['label'] for x in pred]
    return gr.Dropdown.update(choices = pred_object_list, interactive = True)


def Recommend_new(x):
    return x + ": Highest Cosine Similarity"

app = gr.Blocks()

options = ['Placeholder A', 'Placeholder B', 'Placeholder C']
with app:
    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="pil")
        with gr.Column():
            text_options = gr.Dropdown()
            b2 = gr.Button("Provide Additional options")

    b2.click(image_objects, inputs=image_input, outputs=text_options)


app.launch(debug=True)

In [ ]:
# Imports
from PIL import Image, ImageFilter
import numpy as np
from transformers import pipeline
import gradio as gr

model = pipeline("image-segmentation")

def image_objects(image):
    pred = model(image)
    pred_object_list = [x['label'] for x in pred]
    return gr.Dropdown.update(choices = pred_object_list, interactive = True)

def blurr_object(image, object, blur_strength):
    pred = model(image)

    mask_array = np.asarray(pred[0]['mask'])/255
    image_array = np.asarray(image)

    mask_array_three_channel = np.zeros_like(image_array)
    mask_array_three_channel[:,:,0] = mask_array
    mask_array_three_channel[:,:,1] = mask_array
    mask_array_three_channel[:,:,2] = mask_array

    segmented_image = image_array*mask_array_three_channel

    blur_image = np.asarray(image.filter(ImageFilter.GaussianBlur(radius=blur_strength)))
    mask_array_three_channel_invert = 1-mask_array_three_channel
    blur_image_reverse_mask = blur_image*mask_array_three_channel_invert

    return Image.fromarray((blur_image_reverse_mask).astype(np.uint8)+segmented_image.astype(np.uint8))

app = gr.Blocks()


with app:
    with gr.Row():
        with gr.Column():
            gr.Markdown(
            """
            # Input Image
            """)
            image_input = gr.Image(type="pil")
            image_submit_button = gr.Button("Show Objects List")

        with gr.Column():
            gr.Markdown(
            """
            # Found Objects
            """)
            blur_slider = gr.Slider(minimum=1, maximum=10, value=3, label="Adject Blur Strength")
            object_output = gr.Dropdown(label="Select Object From Dropdown")
            object_submit_button = gr.Button("Show Blurred Image")
    
    with gr.Row():
        gr.Markdown(
            """
            # Blurred Image Output
            """)
        image_output = gr.Image()

    image_submit_button.click(fn=image_objects, 
        inputs=image_input, 
        outputs=object_output)

    object_submit_button.click(fn=blurr_object, 
        inputs=[image_input, object_output, blur_slider],
        outputs=image_output)

app.launch(debug=True)

In [ ]:
# Imports
from PIL import Image, ImageFilter
import numpy as np
from transformers import pipeline
import gradio as gr

model = pipeline("image-segmentation")

pred = []

def image_objects(image):
    global pred
    pred = model(image)
    pred_object_list = [str(i)+'_'+x['label'] for i, x in enumerate(pred)]
    return gr.Dropdown.update(choices = pred_object_list, interactive = True)

def blurr_object(image, object, blur_strength):
    object_number = int(object.split('_')[0])
    mask_array = np.asarray(pred[object_number]['mask'])/255
    image_array = np.asarray(image)

    mask_array_three_channel = np.zeros_like(image_array)
    mask_array_three_channel[:,:,0] = mask_array
    mask_array_three_channel[:,:,1] = mask_array
    mask_array_three_channel[:,:,2] = mask_array

    segmented_image = image_array*mask_array_three_channel

    blur_image = np.asarray(image.filter(ImageFilter.GaussianBlur(radius=blur_strength)))
    mask_array_three_channel_invert = 1-mask_array_three_channel
    blur_image_reverse_mask = blur_image*mask_array_three_channel_invert

    return Image.fromarray((blur_image_reverse_mask).astype(np.uint8)+segmented_image.astype(np.uint8))

app = gr.Blocks()


with app:
    with gr.Row():
        with gr.Column():
            gr.Markdown(
            """
            # Input Image
            """)
            image_input = gr.Image(type="pil")
            image_submit_button = gr.Button("Show Objects List")

        with gr.Column():
            gr.Markdown(
            """
            # Found Objects
            """)
            blur_slider = gr.Slider(minimum=1, maximum=10, value=3, label="Adject Blur Strength")
            object_output = gr.Dropdown(label="Select Object From Dropdown")
            object_submit_button = gr.Button("Show Blurred Image")
    
    with gr.Row():
        gr.Markdown(
            """
            # Blurred Image Output
            """)
        image_output = gr.Image()

    image_submit_button.click(fn=image_objects, 
        inputs=image_input, 
        outputs=object_output)

    object_submit_button.click(fn=blurr_object, 
        inputs=[image_input, object_output, blur_slider],
        outputs=image_output)

app.launch(debug=True)

In [6]:
# Imports
from PIL import Image, ImageFilter
import numpy as np
from transformers import pipeline
import gradio as gr
import os

model = pipeline("image-segmentation")

pred = []

def image_objects(image):
    global pred
    pred = model(image)
    pred_object_list = [str(i)+'_'+x['label'] for i, x in enumerate(pred)]
    return gr.Dropdown.update(choices = pred_object_list, interactive = True)

def blurr_object(image, object, blur_strength):
    object_number = int(object.split('_')[0])
    mask_array = np.asarray(pred[object_number]['mask'])/255
    image_array = np.asarray(image)

    mask_array_three_channel = np.zeros_like(image_array)
    mask_array_three_channel[:,:,0] = mask_array
    mask_array_three_channel[:,:,1] = mask_array
    mask_array_three_channel[:,:,2] = mask_array

    segmented_image = image_array*mask_array_three_channel

    blur_image = np.asarray(image.filter(ImageFilter.GaussianBlur(radius=blur_strength)))
    mask_array_three_channel_invert = 1-mask_array_three_channel
    blur_image_reverse_mask = blur_image*mask_array_three_channel_invert

    return Image.fromarray((blur_image_reverse_mask).astype(np.uint8)+segmented_image.astype(np.uint8))



No model was supplied, defaulted to facebook/detr-resnet-50-panoptic (https://huggingface.co/facebook/detr-resnet-50-panoptic)


In [10]:
app = gr.Blocks()


with app:
    with gr.Row():
        with gr.Column():
            gr.Markdown(
            """
            # Input Image
            """)
            image_input = gr.Image(type="pil")
            

        with gr.Column():
            with gr.Row():
                gr.Markdown(
                """
                # Found Objects
                """)
            with gr.Row():
                blur_slider = gr.Slider(minimum=0.5, maximum=10, value=3, label="Adject Blur Strength")
            with gr.Row():
                object_output = gr.Dropdown(label="Select Object From Dropdown")

    
    with gr.Row():
        with gr.Column():
            gr.Markdown(
                """
                # Blurred Image Output
                """)
            image_output = gr.Image()

    image_input.change(fn=image_objects, 
        inputs=image_input, 
        outputs=object_output
        )

    object_output.change(fn=blurr_object, 
        inputs=[image_input, object_output, blur_slider],
        outputs=image_output)

    blur_slider.change(fn=blurr_object, 
        inputs=[image_input, object_output, blur_slider],
        outputs=image_output)

    gr.Examples(
        examples=[
        "test_images/test-1.jpg",
        "test_images/test-2.jpg",
        "test_images/test-3.jpg"
        ],
        fn=image_objects,
        inputs=image_input, 
        outputs=object_output,
        cache_examples=True)

app.launch(debug=True)

Caching examples at: '/media/nybsysml/69a79838-6b45-4f54-b1f0-5846f60d0bf8/nybsysml/koushik/image_segmentation/gradio_cached_examples/134/log.csv'
Running on local URL:  http://127.0.0.1:7860/

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/home/nybsysml/anaconda3/envs/mediapipe3_7/lib/python3.7/site-packages/gradio/routes.py", line 256, in run_predict
    fn_index, raw_input, username, session_state
  File "/home/nybsysml/anaconda3/envs/mediapipe3_7/lib/python3.7/site-packages/gradio/blocks.py", line 599, in process_api
    predictions, duration = await self.call_function(fn_index, processed_input)
  File "/home/nybsysml/anaconda3/envs/mediapipe3_7/lib/python3.7/site-packages/gradio/blocks.py", line 515, in call_function
    block_fn.fn, *processed_input, limiter=self.limiter
  File "/home/nybsysml/anaconda3/envs/mediapipe3_7/lib/python3.7/site-packages/anyio/to_thread.py", line 32, in run_sync
    func, *args, cancellable=cancellable, limiter=limiter
  File "/home/nybsysml/anaconda3/envs/mediapipe3_7/lib/python3.7/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/home/nybsysml/anaconda3/envs/mediapipe3_7/lib/pyt